In [30]:
!pip install qdrant-client sentence-transformers openai pandas

In [31]:
import pandas as pd

# Load your CSV
df = pd.read_csv("/content/100 Best Movies on Netflix.csv")

# Convert each row to a dictionary with text content for RAG
data = []
for _, row in df.iterrows():
    content = (
        f"Title: {row['Movie Title']} | Year: {row['Year']} | "
        f"Director: {row['Director']} | Cast: {row['Cast']} | "
        f"Critics Consensus: {row['Critics Consensus']}"
    )
    data.append({"id": str(row['Rank']), "text": content})

In [32]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('all-MiniLM-L6-v2')  # small and fast
texts = [item['text'] for item in data]
embeddings = embedder.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [58]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance

# Initialize Qdrant
qdrant_client = QdrantClient(":memory:")

# Create collection
if not qdrant_client.collection_exists("movies"):
    qdrant_client.create_collection(
        collection_name="movies",
        vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE)
    )

# Insert points
points = [PointStruct(id=int(item['id']), vector=embeddings[i], payload={"text": item['text']})
          for i, item in enumerate(data)]
qdrant_client.upsert(collection_name="movies", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
def retrieve_context(query, top_k=3):
    q_vec = embedder.encode([query])[0]
    hits = qdrant_client.search(collection_name="movies", query_vector=q_vec, limit=top_k)
    return " ".join([hit.payload["text"] for hit in hits])

In [61]:
#!/usr/bin/env python3
from openai import OpenAI
openai_client = OpenAI(
    base_url="https://api.openai.com/v1", # "http://<Your api-server IP>:port"
    api_key = "api-key"
)

# Example question
question = "Tell me about the movie How to Train Your Dragon"

# Retrieve top-k relevant context from your Qdrant embeddings
context = retrieve_context(question, top_k=3)  # make sure retrieve_context() is defined as before

# Build prompt with context
prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer concisely:"

# Send request
completion = openai_client.chat.completions.create(
    model="gpt-4o-mini",  # smaller, cheaper, fewer tokens
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant specialized in movies."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.5,  # concise answer
    max_tokens=50     # limit token usage
)

print(completion.choices[0].message)

/tmp/ipython-input-3802095976.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant_client.search(collection_name="movies", query_vector=q_vec, limit=top_k)


ChatCompletionMessage(content='"How to Train Your Dragon" is a 2010 animated film directed by Christopher Sanders. The story follows a young Viking named Hiccup, voiced by Jay Baruchel, who befriends a dragon named Toothless, defying his tribe', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
